# Step 1: reading in each day's data from the raw txt from the PDF

In [7]:
import copy
from enum import Enum
import re
import doctest
import json

In [1]:
filelines = []

with open('./input/Lectionary 2022.txt', 'r') as txtfile:
    filelines = txtfile.readlines()
    
len(filelines)

11379

In [3]:
class Section(Enum):
    UNKNOWN = 0,
    MONTH = 1
    DAY = 2,
    DAILY_EUCHARISTIC_L = 3,
    FOR_ALL_SAINTS = 4,
    MORNING_PRAYER = 5,
    EVENING_PRAYER = 6,
    REVISED_COMMON_L = 7,
    OTHER_READINGS_MORNING = 8,
    OTHER_READINGS_EVENING = 9


def isDaySection(line, currentSection):
    '''
    >>> isDaySection("2V", Section.DAY)
    True
    '''
    match = re.search("^(SUN|MON|TUE|WED|THU|FRI|SAT)\\b", line)
    if match:
        return True

    match = re.search("^[0-9]+(W|G|V|v|w|r|R| )*$", line)
    isDayNumber = match and currentSection == Section.DAY
    return True if isDayNumber else False

def isMonthSection(line):
    '''
    >>> isMonthSection("NOVEMBER 2020")
    True
    '''
    match = re.search("^(JANUARY|FEBRUARY|MARCH|APRIL|MAY|JUNE|JULY|AUGUST|SEPTEMBER|OCTOBER|NOVEMBER|DECEMBER) ", line)
    return True if match else False

def isOtherReadingsMorning(line, previousLine):
    '''
    >>> isOtherReadingsMorning("Morning", "Other Readings")
    True
    '''
    if(re.search("^Other Readings", previousLine) and re.search("^Morning", line)):
        return True 
    return False

def isOtherReadingsEvening(line, previousLine):
    '''
    >>> isOtherReadingsEvening("Evening", "Other Readings")
    True
    '''
    if(re.search("^Other Readings", previousLine) and re.search("^Evening", line)):
        return True 
    return False
    
def getSection(line, previousLine = None, currentSection = Section.UNKNOWN):
    '''
    >>> getSection("NOVEMBER 2020", None)
    <Section.MONTH: 1>
    '''
    if(isDaySection(line, currentSection)):
        return Section.DAY
    if(isMonthSection(line)):
        return Section.MONTH
    if(isOtherReadingsMorning(line, previousLine)):
        return Section.OTHER_READINGS_MORNING
    if(isOtherReadingsEvening(line, previousLine)):
        return Section.OTHER_READINGS_EVENING
    
    return Section.UNKNOWN
   
    
doctest.testmod()

TestResults(failed=0, attempted=7)

In [3]:
days = []

def goThroughFile():
    previousLine = None
    currentSection = Section.MONTH
    widerSection = None
    
    currentMonth = "NOVEMBER 2020"
    currentDayOfWeek = "SAT"
    currentDay = "28"
    saints = None
    
    otherReadingsMorning = []
    otherReadingsEvening = []
    
    for line in filelines:
        
        line = line.replace('\n', '')     
        newSection = getSection(line, previousLine, currentSection)
        #print(f"{newSection} {line} ({widerSection})")
           
        hitStartOfNextDay = newSection == Section.MONTH or (newSection == Section.DAY and currentSection != Section.MONTH and currentSection != Section.DAY)
        if(hitStartOfNextDay and currentDayOfWeek):
            #summarise
            dayData = {
                'date': f"{currentDayOfWeek} {currentDay} {currentMonth}",
                'dayDescription': saints,
                'otherReadingsMorning':otherReadingsMorning,
                'otherReadingsEvening':otherReadingsEvening
            }       
            days.append(dayData)
            
            #clear the data
            currentDayOfWeek = None
            currentDay = None
            otherReadingsMorning = []
            otherReadingsEvening = []      
        
        if(newSection == Section.MONTH):  
            currentMonth = line
            widerSection = None
        elif (newSection == Section.DAY and currentSection != Section.DAY):
            currentDayOfWeek = line[0:3]
            saints = line[3:]
            widerSection = Section.DAY
        elif (newSection == Section.DAY and currentSection == Section.DAY):
            match = re.search("^[0-9]+", line)
            if(match):
                currentDay = match.group(0) #the whole match
            widerSection = None
        elif (newSection == Section.OTHER_READINGS_MORNING):
            widerSection = Section.OTHER_READINGS_MORNING
        elif (newSection == Section.OTHER_READINGS_EVENING):
            widerSection = Section.OTHER_READINGS_EVENING
        elif (newSection == Section.UNKNOWN and widerSection == Section.OTHER_READINGS_MORNING and line != "Other Readings"):
            otherReadingsMorning.append(line)
        elif (newSection == Section.UNKNOWN and widerSection == Section.OTHER_READINGS_EVENING):
            otherReadingsEvening.append(line)
          
        currentSection = widerSection if widerSection else newSection
        previousLine = line
              
goThroughFile()

In [4]:
def checkDate(currentDayDate, previousDayDate):
    daysOfWeek = ["MON", "TUE", "WED", "THU", "FRI", "SAT", "SUN"]

    # check for None
    if("None" in currentDayDate):
        return False
    
    dayComponents = currentDayDate.split(" ")    
    if(len(dayComponents) != 4):
        return False
    
    if not previousDayDate:
        return True
    
    previousDayComponents = previousDayDate.split(" ")
    
    # check day
    try:
        difference = int(dayComponents[1]) - int(previousDayComponents[1])
        if difference == 1 or ( dayComponents[1] == "1" and dayComponents[2] != previousDayComponents[2]):
            return True
        else:
            # if today happens more than once and yesterday does too, then we know we have an alternative set of days
            countOfToday = sum([day['date'] == currentDayDate for day in days])
            countOfPreviousDay = sum([day['date'] == previousDayDate for day in days])
            
            return countOfToday == countOfPreviousDay and countOfToday > 1
            
    except ValueError:
        return False
    
    # check day of week
    dayOfWeek = dayComponents[0]
    previousDayOfWeek = previousDayComponents[0]
    difference = daysOfWeek.index(dayOfWeek) - daysOfWeek.index(previousDayOfWeek)
    if(difference not in [1, -6]):
        return False
        
    return True
    

for index, value in enumerate(days):
    dayDate = days[index]['date']
    previousDayDate = days[index - 1]['date'] if index > 0 else None
    
    warningPrefix = '\033[91m'
    okPrefix = '\033[92m'
    
    isOk = checkDate(dayDate, previousDayDate)
    if isOk:
        print(f"{okPrefix}{dayDate}")
    else:
        print(f"{warningPrefix}{dayDate}")
    
    

SAT 28 NOVEMBER 2020
SUN 29 NOVEMBER 2020
MON 30 NOVEMBER 2020
TUE 1 DECEMBER 2020
WED 2 DECEMBER 2020
THU 3 DECEMBER 2020
FRI 4 DECEMBER 2020
SAT 5 DECEMBER 2020
SUN 6 DECEMBER 2020
MON 7 DECEMBER 2020
TUE 8 DECEMBER 2020
WED 9 DECEMBER 2020
THU 10 DECEMBER 2020
FRI 11 DECEMBER 2020
SAT 12 DECEMBER 2020
SUN 13 DECEMBER 2020
MON 14 DECEMBER 2020
TUE 15 DECEMBER 2020
WED 16 DECEMBER 2020
THU 17 DECEMBER 2020
FRI 18 DECEMBER 2020
SAT 19 DECEMBER 2020
SUN 20 DECEMBER 2020
MON 21 DECEMBER 2020
TUE 22 DECEMBER 2020
WED 23 DECEMBER 2020
THU 24 DECEMBER 2020
FRI 25 DECEMBER 2020
SAT 26 DECEMBER 2020
SUN 27 DECEMBER 2020
MON 28 DECEMBER 2020
TUE 29 DECEMBER 2020
WED 30 DECEMBER 2020
THU 31 DECEMBER 2020
FRI 1 JANUARY 2021
FRI 1 JANUARY 2021
SAT 2 JANUARY 2021
SUN 3 JANUARY 2021
MON 4 JANUARY 2021
TUE 5 JANUARY 2021
WED 6 JANUARY 2021
THU 7 JANUARY 2021
FRI 8 JANUARY 2021
SAT 9 JANUARY 2021
SUN 3 JANUARY 2021
MON 4 JANUARY 2021
TUE 5 JANUARY 2021
WED 6 JANUARY 2021
THU 7 JANUARY 2021
FRI 8 JANU

## Stop
Only continue once the above statement prints all the dates correctly.  If it looks like a rare typo (like Sun instead of SUN), just tweak the document.  If it looks like a common thing, try change the script.

In [4]:
# this is just a script to inspect the data for a particular day
foundDay = [day for day in days if day['date'] == "SUN 27 DECEMBER 2020"][0]
print(foundDay)

IndexError: list index out of range

# Step 2: Attempting to process the morning and evening readings programatically before finishing manually

In [5]:
def removeOrAndTrim(reading):
    result = reading.replace(" or ", "")
    result = result.replace(" or", "")
    result = result.replace("or ", "")
    return result.strip()

def resolveAlternatives(list):   
    apocryphal = ["Tobit", "1 Macc", "2 Macc", "Sirach", "Baruch", "Wisd"]
    result = []
    for book in apocryphal:
        if book in list[1]:
            result = [list[0], list[2], list[3], list[1]]
        elif book in list[2]:
            result = [list[0], list[1], list[3], list[2]]

    return [removeOrAndTrim(reading) for reading in result]

def looksLikeScripture(reading):
    script_regex = "^([1-2] )?[A-Za-z]+ [\[\]\(\)0-9]+([0-9\-–;:,endab \(\)\*]+)?$"
    return re.search(script_regex, reading) is not None

def removeExtras(list):   
    '''
    >>> removeExtras(['Ps 68:1-19', 'Mal 3:1-4;4', 'Phil 4:4-7', 'Celebrating Common Prayer: Form 3 during Advent', 'Collect for the Third Sunday of Advent'])
    ['Ps 68:1-19', 'Mal 3:1-4;4', 'Phil 4:4-7']
    '''
    
    filtered_list = [removeOrAndTrim(scripture) for scripture in list]
    
    if not looksLikeScripture(filtered_list[0]):
        return [f"error: expected scripture in '{filtered_list[0]}'"] + filtered_list
    elif not looksLikeScripture(filtered_list[1]):
        return [f"error: expected scripture in '{filtered_list[1]}'"] + filtered_list
    elif not looksLikeScripture(filtered_list[2]):
        return [f"error: expected scripture in '{filtered_list[2]}'"] + filtered_list
    
    filtered_list = [scripture for scripture in filtered_list if looksLikeScripture(scripture)]
    
    if len(filtered_list) == 3:
        return filtered_list
    if len(filtered_list) == 4:
        return resolveAlternatives(filtered_list)
    
    return ["error: couldn't resolve down to 3"] + filtered_list
    
                      
        
def processOtherReadings(day):
    
    key = ""
    result = copy.deepcopy(day)
    
    # morning
    key = "otherReadingsMorning"
    unprocessedReadingsList = day[key]   
    if len(unprocessedReadingsList) == 3:
        result[key] = unprocessedReadingsList
    elif len(unprocessedReadingsList) == 4:
        result[key] = resolveAlternatives(unprocessedReadingsList)
    elif len(unprocessedReadingsList) > 4:
        result[key] = removeExtras(unprocessedReadingsList)
    else:
        result[key] = []
        
    # evening
    key = "otherReadingsEvening"
    unprocessedReadingsList = day[key]   
    if len(unprocessedReadingsList) == 3:
        result[key] = unprocessedReadingsList
    elif len(unprocessedReadingsList) == 4:
        result[key] = resolveAlternatives(unprocessedReadingsList)
    elif len(unprocessedReadingsList) > 4:
        result[key] = removeExtras(unprocessedReadingsList)
    else:
        result[key] = []
        
    return result

      
def processDays():
    
    result = []
    for day in days: 
        processed_day = processOtherReadings(day)

        if not "SUN" in day["date"]:
            processed_day["otherReadingsMorning"] = []
            processed_day["otherReadingsEvening"] = []
        
        result.append(processed_day)
        
    return result 
    
doctest.testmod()

TestResults(failed=0, attempted=6)

In [6]:
# debug a particular day
foundDay = [day for day in days if day['date'] == "MON 11 JANUARY 2021"][0]
processOtherReadings(foundDay)

IndexError: list index out of range

In [7]:
processedDays = processDays()

for day in processedDays:
    morningR = day['otherReadingsMorning']
    eveningR = day['otherReadingsEvening']
    
    print(f"{day['date']}%%{'; '.join(morningR)}%%{'; '.join(eveningR)}")

SAT 28 NOVEMBER 2020%%%%
SUN 28 NOVEMBER 2021%%Ps 44; Isa 51:4-11; Rom 13:11-14%%Ps 9*; Joel 3:9-21; Rev 14:13–15:4
MON 29 NOVEMBER 2021%%%%
TUE 30 NOVEMBER 2021%%%%
WED 1 DECEMBER 2021%%%%
THU 2 DECEMBER 2021%%%%
FRI 3 DECEMBER 2021%%%%
SAT 4 DECEMBER 2021%%%%
SUN 5 DECEMBER 2021%%Ps 80; Isa 64:1-7; Matt 11:2-11%%Ps 75,(76); Isa 40:1-11; Luke 1:1-25
MON 6 DECEMBER 2021%%%%
TUE 7 DECEMBER 2021%%%%
WED 8 DECEMBER 2021%%%%
THU 9 DECEMBER 2021%%%%
FRI 10 DECEMBER 2021%%%%
SAT 11 DECEMBER 2021%%%%
SUN 12 DECEMBER 2021%%Ps 12,14; Isa 25:1-9; 1 Cor 4:1-5%%Ps 50:1-6;(62); Isa 35; Luke 1:57-66,
MON 13 DECEMBER 2021%%%%
TUE 14 DECEMBER 2021%%%%
WED 15 DECEMBER 2021%%%%
THU 16 DECEMBER 2021%%%%
FRI 17 DECEMBER 2021%%%%
SAT 18 DECEMBER 2021%%%%
SUN 19 DECEMBER 2021%%Ps 144; Isa 32:1-8; Rev 22:6-21%%Ps 123;(131); Isa 10:33–11:10; Matt 1:18-25
MON 20 DECEMBER 2021%%%%
TUE 21 DECEMBER 2021%%%%
WED 22 DECEMBER 2021%%%%
THU 23 DECEMBER 2021%%%%
FRI 24 DECEMBER 2021%%%%
SAT 25 DECEMBER 2021%%%%
SUN 26 

## Stop
Take the output of the last step, paste it into a text file, and go through each sunday, fixing them up so there are 3 readings in each section, and save the file in `input` folder named `raw_sunday_readings_manually_selected.txt`

I have been:
1. Removing extra psalms in brackets, but if there are multiple main ones, leaving them
2. Keep an eye out for the apochyphal books - the processing this far isn't that great
3. For sundays where there are multiple options, I'm trying to pick the one that normally gets celebrated on that day

# Step 3: Cleaning and formatting the Sunday readings


In [8]:
processed_days = []

processedfilelines = []
with open('./input/raw_sunday_readings_manually_selected_2022.txt', 'r') as txtfile:
    processedfilelines = txtfile.readlines()
    
for line in processedfilelines:
    day_morning_evening = line.replace("\n", "").split("%%")
    processed_days.append({
        "date": day_morning_evening[0], 
        "otherReadingsMorning": day_morning_evening[1], 
        "otherReadingsEvening": day_morning_evening[2]
    })
    

In [9]:
def hydrate_book_names(scripture_list):
    '''
    >>> hydrate_book_names(['Ps 44', 'Isa 2:1-5', 'Luke 12:35-48'])
    ['Psalm 44', 'Isaiah 2:1-5', 'Luke 12:35-48']
    '''
    
    book_replacement_dictionary = {
        '^(Gen )': 'Genesis ',
        '^(Exod |Ex )': 'Exodus ',
        '^(Lev )': 'Leviticus ',
        '^(Num |Numb )': 'Numbers ',
        '^(Deut )': 'Deuteronomy ',
        '^(Josh )': 'Joshua ',
        '^(Judg )': 'Judges ',
        '^(Ruth )': 'Ruth ',
        '^(1 Sam )': '1 Samuel ',
        '^(2 Sam )': '2 Samuel ',
        '^(1 Kgs )': '1 Kings ',
        '^(2 Kgs )': '2 Kings ',
        '^(1 Chr|1 Chron )': '1 Chronicles ',
        '^(2 Chr|1 Chron )': '2 Chronicles ',
        '^(Ezra )': 'Ezra ',
        '^(Neh )': 'Nehemiah ',
        '^(Esth )': 'Esther ',
        '^(Job )': 'Job ',
        '^(Ps )': 'Psalm ',
        '^(Prov )': 'Proverbs ',
        '^(Eccl )': 'Ecclesiastes ',
        '^(Song of Sol )': 'Song of Solomon ',
        '^(Isa )': 'Isaiah ',
        '^(Jer )': 'Jeremiah ',
        '^(Lam )': 'Lamentations ',
        '^(Ezek )': 'Ezekiel ',
        '^(Dan )': 'Daniel ',
        '^(Hos )': 'Hosea ',
        '^(Joel )': 'Joel ',
        '^(Amos )': 'Amos ',
        '^(Obad )': 'Obadiah ',
        '^(Jonah )': 'Jonah ',
        '^(Mic )': 'Micah ',
        '^(Nah )': 'Nahum ',
        '^(Hab )': 'Habakkuk ',
        '^(Zeph )': 'Zephaniah ',
        '^(Hag )': 'Haggai ',
        '^(Zech )': 'Zechariah ',
        '^(Mal )': 'Malachi ',
        '^(Matt )': 'Matthew ',
        '^(Mark )': 'Mark ',
        '^(Luke )': 'Luke ',
        '^(John )': 'John ',
        '^(Acts )': 'Acts ',
        '^(Rom )': 'Romans ',
        '^(1 Cor )': '1 Corinthians ',
        '^(2 Cor )': '2 Corinthians ',
        '^(Gal )': 'Galatians ',
        '^(Eph )': 'Ephesians ',
        '^(Phil )': 'Philippians ',
        '^(Col )': 'Colossians ',
        '^(1 Thess )': '1 Thessalonians ',
        '^(2 Thess )': '2 Thessalonians ',
        '^(1 Tim )': '1 Timothy ',
        '^(2 Tim )': '2 Timothy ',
        '^(Titus )': 'Titus ',
        '^(Phlm |Philem )': 'Philemon ',
        '^(Heb )': 'Hebrews ',
        '^(Jas )': 'James ',
        '^(1 Pet )': '1 Peter ',
        '^(2 Pet )': '2 Peter ',
        '^(1 John )': '1 John ',
        '^(2 John )': '2 John ',
        '^(3 John )': '3 John ',
        '^(Jude )': 'Jude ',
        '^(Rev )': 'Revelation ',
    }
    
    result = []
    for scripture in scripture_list:
        new_scripture = scripture
        for regex in book_replacement_dictionary.keys():
            new_scripture = re.sub(regex, book_replacement_dictionary[regex], new_scripture)
         
        result.append(new_scripture)
    
    return result

def process_joined_readings(joined_readings):
    '''
    >>> process_joined_readings('Ps 44; Isa 2:1-5; Luke 12:35-48')
    'Psalm 44; Isaiah 2:1-5; Luke 12:35-48'
    '''
    
    if joined_readings == '':
        return ''
    
    split_readings = joined_readings.split("; ") 
    split_readings = hydrate_book_names(split_readings)  
    return "; ".join(split_readings)

    
doctest.testmod()

TestResults(failed=0, attempted=7)

In [10]:
def process_again():
    result = []
    for day in processed_days:
        new_day = copy.deepcopy(day)
        new_day['otherReadingsMorning'] = process_joined_readings(new_day['otherReadingsMorning'])
        new_day['otherReadingsEvening'] = process_joined_readings(new_day['otherReadingsEvening'])
        
        print(f"{new_day['date']}%%{new_day['otherReadingsMorning']}%%{new_day['otherReadingsEvening']}")
    
process_again()

SAT 28 NOVEMBER 2020%%%%
SUN 28 NOVEMBER 2021%%Psalm 44; Isaiah 51:4-11; Romans 13:11-14%%Psalm 9; Joel 3:9-21; Revelation 14:13–15:4
MON 29 NOVEMBER 2021%%%%
TUE 30 NOVEMBER 2021%%%%
WED 1 DECEMBER 2021%%%%
THU 2 DECEMBER 2021%%%%
FRI 3 DECEMBER 2021%%%%
SAT 4 DECEMBER 2021%%%%
SUN 5 DECEMBER 2021%%Psalm 80; Isaiah 64:1-7; Matthew 11:2-11%%Psalm 75; Isaiah 40:1-11; Luke 1:1-25
MON 6 DECEMBER 2021%%%%
TUE 7 DECEMBER 2021%%%%
WED 8 DECEMBER 2021%%%%
THU 9 DECEMBER 2021%%%%
FRI 10 DECEMBER 2021%%%%
SAT 11 DECEMBER 2021%%%%
SUN 12 DECEMBER 2021%%Psalm 12,14; Isaiah 25:1-9; 1 Corinthians 4:1-5%%Psalm 50:1-6; Isaiah 35; Luke 1:57-66
MON 13 DECEMBER 2021%%%%
TUE 14 DECEMBER 2021%%%%
WED 15 DECEMBER 2021%%%%
THU 16 DECEMBER 2021%%%%
FRI 17 DECEMBER 2021%%%%
SAT 18 DECEMBER 2021%%%%
SUN 19 DECEMBER 2021%%Psalm 144; Isaiah 32:1-8; Revelation 22:6-21%%Psalm 123; Isaiah 10:33–11:10; Matthew 1:18-25
MON 20 DECEMBER 2021%%%%
TUE 21 DECEMBER 2021%%%%
WED 22 DECEMBER 2021%%%%
THU 23 DECEMBER 2021%%%%

## Step 4: Publishing
1. Copy paste the above into the master sheet "constructing the year" tab (ensuring the dates match up all the way down)
2. Split text to columns on '%%'
3. Delete the extra date column, name the two new columns somethig like "Other Readings Morning/Evening", 
4. Adjust the morning and evening formula to use the new columns for Sundays
5. Copy paste the values of `Morning prayers together` and `Evening prayers together` into the published spreadhseet
6. Download a csv of the published spreadsheet tab, save it in the `input` folder as `all_readings.csv`.  
7. Run it through the `write_data_to_json.ipynb` script to publish the data.